In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import nltk

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.read_csv('../input/jigsaw-toxic-severity-rating/comments_to_score.csv')

In [ ]:
data=pd.read_csv('../input/jigsaw-toxic-severity-rating/validation_data.csv')

In [ ]:
data.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def get_length(text):
    return len(text.split())

In [ ]:
less_toxic_lengths = data['less_toxic'].apply(get_length)

In [ ]:
sns.kdeplot(less_toxic_lengths)

In [ ]:
more_toxic_lengths = data['more_toxic'].apply(get_length)

In [ ]:
sns.kdeplot(more_toxic_lengths)

In [ ]:
!pip install textstat
!pip install pyicu
!pip install pycld2
import textstat
from polyglot.detect import Detector
def detect_language(text):
    return Detector("".join(x for x in text if x.isprintable()), quiet=True).languages[0].name

In [ ]:
all_coment= data.less_toxic.append(data.more_toxic)
all_coments=all_coment.value_counts().index.values

In [ ]:
languages = [detect_language(comment) for comment in all_coments]
language_df = pd.DataFrame({
    'text': all_coments,
    'language': languages
})

language_df['language'].value_counts().to_frame().head(15)

In [ ]:
length_diff=more_toxic_lengths -less_toxic_lengths

In [ ]:
sns.kdeplot(length_diff)

In [ ]:
import string

punctuation_count = lambda l1,l2: sum([1 for x in l1 if x in l2])
data['less_toxic_punctuation_count'] = data.less_toxic.apply(lambda comment: punctuation_count(comment,set(string.punctuation)))
data['more_toxic_punctuation_count'] = data.more_toxic.apply(lambda comment: punctuation_count(comment,set(string.punctuation)))

# exclamation mark count
exclmataion_count = lambda l1: sum([1 for x in l1 if x =='!'])
data['less_toxic_exclmataion_count'] = data.less_toxic.apply(lambda comment: exclmataion_count(comment))
data['more_toxic_exclmataion_count'] = data.more_toxic.apply(lambda comment: exclmataion_count(comment))

punctuation_value_counts = data.apply(lambda row: 1 if row.more_toxic_punctuation_count > row.less_toxic_punctuation_count else 0, axis=1).value_counts()
exclamation_value_counts = data.apply(lambda row: 1 if row.more_toxic_exclmataion_count >= row.less_toxic_exclmataion_count else 0, axis=1).value_counts()

fig = plt.figure(figsize=(12,8))

ax1 = fig.add_subplot(121)
ax1.pie(punctuation_value_counts, labels = ['Less toxic', 'More toxic'], colors = ['skyblue', '#ff6666'])
ax1.set_title('Puncutaion count')

ax2 = fig.add_subplot(122)
ax2.pie(exclamation_value_counts, labels = ['Less toxic', 'More toxic'], colors = ['skyblue', '#ff6666'])
ax2.set_title('Exclamation count')

plt.show()


In [ ]:
repseation=data[['less_toxic','more_toxic']].value_counts().values

In [ ]:
repetaion_df=pd.DataFrame({'unsync_repetaions':repseation})
sns.barplot(y=repetaion_df['unsync_repetaions'].value_counts().values,x=repetaion_df['unsync_repetaions'].value_counts().index)

In [ ]:
data['paired']=data['less_toxic']+':'+data['more_toxic']

In [ ]:
paried_df=pd.DataFrame({'counts':data['paired'].value_counts().values})

In [ ]:
sns.barplot(y=paried_df['counts'].value_counts().values,x=paried_df['counts'].value_counts().index)

In [ ]:
less_toxic_reading_score = data.less_toxic.apply(lambda comment: textstat.flesch_reading_ease(comment)).mean()
more_toxic_reading_score = data.more_toxic.apply(lambda comment: textstat.flesch_reading_ease(comment)).mean()

plt.figure(figsize=(8, 6))
plt.bar(['Less toxic', 'More toxic'], [less_toxic_reading_score, more_toxic_reading_score], alpha=0.5, width=0.6)
plt.ylabel('Readability score')
plt.title('Flesch Readability score')
plt.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
less_toxic_vectorizer=CountVectorizer(stop_words='english')
more_toxic_vectorizer=CountVectorizer(stop_words='english')

In [ ]:
def remove_non_english(sent):
    words = set(nltk.corpus.words.words())
    output=" ".join(w for w in nltk.wordpunct_tokenize(sent) \
         if w.lower() in words or not w.isalpha())
    return output

In [ ]:
data['less_toxic']=data['less_toxic'].apply(remove_non_english)
data['more_toxic']=data['more_toxic'].apply(remove_non_english)

In [ ]:
less_toxic_vectorizer.fit(data['less_toxic'])

In [ ]:
more_toxic_vectorizer.fit(data['more_toxic'])

In [ ]:
less_vocab=less_toxic_vectorizer.vocabulary_
more_vocab = more_toxic_vectorizer.vocabulary_

In [ ]:
import operator
def plot_top_10(values):
    sorted_d = dict(sorted(values.items(), key=operator.itemgetter(1),reverse=True))
    top_words=[]
    counts=[]
    for word,count in sorted_d.items():
        top_words.append(word.lower())
        counts.append(count)
        if len(counts)>=10:
            break
    sns.barplot(x=top_words,y=counts)
    plt.show()
    return top_words,counts

In [ ]:
plot_top_10(less_vocab)